# Role-Based Access Control (RBAC) Reduction

RBAC is a method of access control that restricts system access to authorized users based on their roles within an organization. In RBAC, permissions are assigned to roles, rather than to individual users. Users are then assigned to roles based on their job responsibilities or other criteria. This allows for more efficient management of access control, since permissions can be assigned to roles once, rather than to each individual user.

**PROBLEM:** Over time, in a large enterprise, the number of roles grows due to the appearance of new resources. Therefore, periodically cleaning the list is required. In this notebook, we propose an approach of how to do this, and compare it with using network analysis.


## Prerequisites


In [6]:
import pandas as pd
import numpy as np
import random
import math
import networkx as nx
from collections import OrderedDict
from collections import namedtuple
from collections import defaultdict
from tqdm import tqdm
from datasketch import HNSW
from enum import Enum

## Generate synthetic data

Here we create a function to generate synthetic data to ensure reproducibility. However, this synthetic data can be easily replaced by any other data set which contains relationships.

### Define generating function

In [3]:
def generate_business_roles(num_roles, num_people, percent_same, max_members, max_roles_in_group):
    
    """
    Generates a DataFrame simulating a business role membership structure.

    Parameters:
    num_roles: int
        The total number of business roles to generate.
        Each role is named as "BR1", "BR2", ..., "BRn".

    num_people: int
        The total number of people to consider for assignment to business roles.
        Each person is identified as "P1", "P2", ..., "Pn".

    percent_same: int
        The percentage of business roles that will have identical membership.
        This dictates how many roles will share the exact same set of members.
        The value should be between 0 and 100.

    max_members: int
        The maximum number of members that can be assigned to a single business role.
        Actual number of members in a role will vary randomly between 0 and max_members.

    max_roles_in_group: int
        The maximum number of business roles that can form a group with identical members.
        Determines the largest possible group of roles sharing the same members.

    Returns:
    DataFrame
        A pandas DataFrame with two columns: 'business_role' and 'person'.
        'business_role' contains the business role IDs.
        'person' contains the IDs of people assigned to each business role.
        Each row represents a person's membership in a specific business role.

    Example usage:
    df = generate_business_roles(10, 50, 20, 5, 3)
    This creates a structure with 10 business roles, 50 people, 20% of roles sharing members,
    a maximum of 5 members per role, and a maximum of 3 roles in a group with shared members.
    """

    # Generate business roles and people IDs
    business_roles = ['BR' + str(i+1) for i in range(num_roles)]
    people = ['P' + str(i+1) for i in range(num_people)]

    # Determine the number of roles in the largest group
    max_group_size = min(max_roles_in_group, num_roles)

    # Determine the number of roles with identical members
    num_same = int(num_roles * percent_same / 100)

    # Assign members to these roles
    same_member_roles = {}
    for i in range(0, num_same, max_group_size):
        members = random.sample(people, random.randint(0, max_members))
        for role in business_roles[i:i + max_group_size]:
            same_member_roles[role] = members

    # Generate data for each business role
    data = []
    for role in business_roles:
        if role in same_member_roles:
            members = same_member_roles[role]
        else:
            members = random.sample(people, random.randint(0, max_members))

        for person in members:
            data.append({'business_role': role, 'person': person})

    return pd.DataFrame(data)

### Generate data

Below, we have three proposed options for a synthetic dataset to run the code below:

- *Minimum Example*: Small set with the purpose of ensuring the whole notebook can be easily run.
- *Minimum Example Demonstrating Method Differences*: Altough still small compared with real data, this dataset is sufficient to highlight differences between methodologies. Run this first; the network analysis with the realistic dataset takes considerably more time.
- *Realistic example*: This example emulates our real world scenario. It takes a considerable time to run the network analysis on this example, however our proposed methodology runs it quite efficiently.

In [4]:
# Minimum example for code testing
# df = generate_business_roles(num_roles = 100, num_people = 50, percent_same = 10, max_members = 10, max_roles_in_group = 5)

# Minimum example which already shows differences between methods
df = generate_business_roles(num_roles = 5000, num_people = 500, percent_same = 10, max_members = 20, max_roles_in_group = 10)

# Realist example seen in real data
# df = generate_business_roles(num_roles = 50000, num_people = 80000, percent_same = 10, max_members = 250, max_roles_in_group = 150)

df.head()

,business_role,person
0,BR1,P111
1,BR1,P333
2,BR1,P320
3,BR1,P366
4,BR1,P209


## Compare algorithms

Set the maximum number of extra members in a business role to still consider grouping it with another.

This is used in both algorithms.

In [5]:
max_extra_members = 0

### Network analysis

#### Convert dataframe to dictionaries of nodes and edges

In [7]:
Node = namedtuple('Node', ['label', 'node_type'])

class NodeType(Enum):
    USER = 1
    ROLE = 2

nodes_dict = OrderedDict()

for index, row in df.iterrows():
    business_role = row['business_role']
    person = row['person']
    if person not in nodes_dict:  # Check for duplicate nodes before adding
        nodes_dict[person] = Node(label=person, node_type=NodeType.USER)
    if business_role not in nodes_dict:
        nodes_dict[business_role] = Node(label=business_role, node_type=NodeType.ROLE)

edges = []
for index, row in df.iterrows():
    person = row['person']
    business_role = row['business_role']
    edges.append((nodes_dict[person], nodes_dict[business_role]))
    
all_nodes = nodes_dict.values()

#### Find business role pairs

##### Build graph network of business roles connected through employees

In [8]:
nx_graph = nx.Graph()

for node in all_nodes:
    nx_graph.add_node(
        node, 
        subset=node.node_type.value
    )

for edge in edges:
    nx_graph.add_edge(*edge)
    
adj_matrix = nx.to_numpy_array(nx_graph, dtype=np.int8)

# node indicies are required later for selecting corresponding rows and columns from the adjacency matrix
nodes_indicies = {
    NodeType.USER : [],
    NodeType.ROLE : [],
}

role_nodes = []

for i, node in enumerate(nx_graph.nodes()):
    nodes_indicies[node.node_type].append(i)
    if node.node_type == NodeType.ROLE:
        role_nodes.append(node)

role_user_matrix = adj_matrix[np.ix_(nodes_indicies[NodeType.USER], nodes_indicies[NodeType.ROLE])].T

manhattan_distance = lambda x, y: np.sum(np.abs(x - y))

role_user_index = HNSW(distance_func=manhattan_distance)

for n, ru in zip(role_nodes, role_user_matrix):
    role_user_index.insert(n, ru)


##### Identify pairs of business roles which have the same employees

In [9]:
max_k = math.ceil(len(nodes_indicies[NodeType.ROLE])/2)

result_list = []

for n, ru in zip(role_nodes, role_user_matrix):
    query_result = role_user_index.query(ru, k=max_k)
    for (l, dst) in query_result:
        if dst <= max_extra_members: 
            if n != l:
                emps = df[df['business_role']==n.label]['person'].nunique()
                result_list.append((emps, sorted(list([n.label, l.label])), dst))
        else:
            break

result_df_network = pd.DataFrame(result_list, columns=['emps', 'brs', 'dist'])
result_df_network.head(2)

,emps,brs,dist
0,12,"[BR1, BR9]",0
1,12,"[BR1, BR8]",0


#### Combine business role pairs into larger connected groups

In [10]:
# Function to find common groups
def find_common_groups(df):
    unique_groups = []
    for brs in df['brs']:
        found = False
        for group in unique_groups:
            if any(br in brs for br in group):
                group.update(brs)
                found = True
                break
        if not found:
            unique_groups.append(set(brs))
    return unique_groups

# Finding common groups
unique_groups = find_common_groups(result_df_network)

# Creating the aggregated DataFrame
aggregated_data = []
for grp_number, group in enumerate(unique_groups, start=0):
    matching_rows = result_df_network[result_df_network['brs'].apply(lambda x: any(br in group for br in x))]
    num_mbrs = matching_rows['emps'].max()
    dst = matching_rows['dist'].max()
    aggregated_data.append([grp_number, num_mbrs, sorted(list(group)), dst])

result_df_network = pd.DataFrame(aggregated_data, columns=['grp', 'num_mbrs', 'brs', 'dist'])
result_df_network['num_brs'] = result_df_network['brs'].apply(lambda x: len(x))

result_df_network.head(2)


,grp,num_mbrs,brs,dist,num_brs
0,0,12,"[BR1, BR10, BR2, BR3, BR4, BR5, BR6, BR7, BR8,...",0,10
1,1,2,"[BR11, BR12, BR13, BR14, BR15, BR16, BR17, BR1...",0,10


### Co-occurrence algorithm

#### Create co-occurrence matrix

In [11]:
# Group by person and create a list of business roles for each person
grouped_brs = df.groupby('person')['business_role'].apply(list)

# Create a defaultdict to hold the br combination counts
br_combination_counts_dict = defaultdict(int)

# Iterate through the grouped brs and increment the counts for each combination of brs
for person_brs in tqdm(grouped_brs):
    for br1 in person_brs:
        for br2 in person_brs:
            br_combination_counts_dict[(br1, br2)] += 1

# Convert the defaultdict to a DataFrame
br_combination_counts = pd.DataFrame.from_dict(br_combination_counts_dict, orient='index', columns=['count'])

# Reset the index to separate the br names into separate columns
br_combination_counts.reset_index(inplace=True)
br_combination_counts[['br_1', 'br_2']] = pd.DataFrame(br_combination_counts['index'].tolist(), index=br_combination_counts.index)
br_combination_counts.drop(columns=['index'], inplace=True)

# Pivot the DataFrame to create the final matrix
br_combination_counts = br_combination_counts.pivot(index='br_1', columns='br_2', values='count').fillna(0)

100%|██████████| 500/500 [00:01<00:00, 367.34it/s]


#### Find business role groups

In [12]:
max_dist = max_extra_members

# List to store the results
result_list = []
done_brs = []
br_ent_list = df[['business_role']].drop_duplicates()

for col in tqdm(range(br_combination_counts.shape[0])):
    if not (br_combination_counts.iloc[col::,col].index[0] in done_brs):
        vec = br_combination_counts.iloc[col::,col]
        value = vec[0]
        diag_vec = np.diag(br_combination_counts)[col::][vec==value]
        num_mbrs = max(diag_vec[np.abs(diag_vec - value) <= max_dist])
        dst = num_mbrs - min(diag_vec[np.abs(diag_vec - value) <= max_dist])
        vec = vec[vec==value][np.abs(diag_vec - value) <= max_dist]
        result_list.append((col, int(max(vec)), sorted(list(vec.index)), int(dst)))
        done_brs.extend(list(vec.index))

# Create a DataFrame from the result list
result_df = pd.DataFrame(result_list, columns=['grp', 'num_mbrs', 'brs', 'dist'])

result_df['num_brs'] = result_df['brs'].apply(lambda x: len(x))

result_df = result_df.reset_index(drop=True)
result_df['grp'] = result_df.index

# Print the result
result_df.head(2)

  0%|          | 0/4788 [00:00<?, ?it/s]/var/folders/5g/pznhl14d595934c1gmn58zsm0000gp/T/ipykernel_44623/3558056359.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value = vec[0]
100%|██████████| 4788/4788 [00:00<00:00, 5017.51it/s]


,grp,num_mbrs,brs,dist,num_brs
0,0,12,"[BR1, BR10, BR2, BR3, BR4, BR5, BR6, BR7, BR8,...",0,10
1,1,15,"[BR100, BR91, BR92, BR93, BR94, BR95, BR96, BR...",0,10


### Compare results

#### Co-occurrence's Results

In [13]:
result_df_cooc = result_df[result_df['num_brs'] > 1]
print(f'Number of groups: {result_df_cooc.shape[0]}')
display(result_df_cooc.head(10))

Number of groups: 90


,grp,num_mbrs,brs,dist,num_brs
0,0,12,"[BR1, BR10, BR2, BR3, BR4, BR5, BR6, BR7, BR8,...",0,10
1,1,15,"[BR100, BR91, BR92, BR93, BR94, BR95, BR96, BR...",0,10
11,11,2,"[BR101, BR102, BR103, BR104, BR105, BR106, BR1...",0,10
17,17,1,"[BR1015, BR2392, BR3147]",0,3
82,82,1,"[BR1087, BR3015]",0,2
95,95,2,"[BR11, BR12, BR13, BR14, BR15, BR16, BR17, BR1...",0,10
106,106,20,"[BR111, BR112, BR113, BR114, BR115, BR116, BR1...",0,10
160,160,1,"[BR1169, BR4139]",0,2
197,197,5,"[BR121, BR122, BR123, BR124, BR125, BR126, BR1...",0,10
225,225,1,"[BR1237, BR3595]",0,2


#### Network Results

In [14]:
print(f'Number of groups: {result_df_network.shape[0]}')
display(result_df_network.head(10))

Number of groups: 56


,grp,num_mbrs,brs,dist,num_brs
0,0,12,"[BR1, BR10, BR2, BR3, BR4, BR5, BR6, BR7, BR8,...",0,10
1,1,2,"[BR11, BR12, BR13, BR14, BR15, BR16, BR17, BR1...",0,10
2,2,9,"[BR21, BR22, BR23, BR24, BR25, BR26, BR27, BR2...",0,10
3,3,4,"[BR31, BR32, BR33, BR34, BR35, BR36, BR37, BR3...",0,10
4,4,13,"[BR41, BR42, BR43, BR44, BR45, BR46, BR47, BR4...",0,10
5,5,13,"[BR51, BR52, BR53, BR54, BR55, BR56, BR57, BR5...",0,10
6,6,12,"[BR61, BR62, BR63, BR64, BR65, BR66, BR67, BR6...",0,10
7,7,16,"[BR71, BR72, BR73, BR74, BR75, BR76, BR77, BR7...",0,10
8,8,5,"[BR81, BR82, BR83, BR84, BR85, BR86, BR87, BR8...",0,10
9,9,15,"[BR100, BR91, BR92, BR93, BR94, BR95, BR96, BR...",0,10


#### Compare groups from both approaches

Groups found by the network method, not found by the co-occurrence method

In [15]:
df_diff_network = result_df_network[~result_df_network['brs'].isin(result_df_cooc['brs'])]
print(f'Number of groups in the Network\'s DF not in the Co-ocurrence\'s: {df_diff_network.shape[0]}')
display(df_diff_network)

Number of groups in the Network's DF not in the Co-ocurrence's: 0


,grp,num_mbrs,brs,dist,num_brs


Groups found by the co-occurrence method, not found by the network method

In [16]:
df_diff_cooc = result_df_cooc[~result_df_cooc['brs'].isin(result_df_network['brs'])]
print(f'Number of groups in the Co-occurrence\'s DF not in the Network\'s: {df_diff_cooc.shape[0]}')
display(df_diff_cooc)

Number of groups in the Co-occurrence's DF not in the Network's: 34


,grp,num_mbrs,brs,dist,num_brs
106,106,20,"[BR111, BR112, BR113, BR114, BR115, BR116, BR1...",0,10
294,294,17,"[BR131, BR132, BR133, BR134, BR135, BR136, BR1...",0,10
484,484,18,"[BR151, BR152, BR153, BR154, BR155, BR156, BR1...",0,10
581,581,18,"[BR161, BR162, BR163, BR164, BR165, BR166, BR1...",0,10
620,620,1,"[BR1649, BR3636]",0,2
678,678,20,"[BR171, BR172, BR173, BR174, BR175, BR176, BR1...",0,10
747,747,1,"[BR1781, BR1838]",0,2
875,875,1,"[BR1913, BR4528]",0,2
909,909,1,"[BR1948, BR3479]",0,2
980,980,1,"[BR2023, BR2569]",0,2
